In [1]:
# sudo apt-get update
# sudo apt-get install graphviz graphviz-dev


In [2]:
# from IPython.display import Image, display
# from graphviz import Digraph
# 
# # Criando o diagrama de decisão para recomendação com Cold Start e Recência
# dot = Digraph(comment='Sistema de Recomendação: Exploração vs. Exploitação', format='png')
# 
# # Nós principais
# dot.node('A', 'Novo Usuário?')
# dot.node('B', 'Exploração (Cold Start)')
# dot.node('C', 'Exibir Itens Populares/Novos')
# dot.node('D', 'Coleta de Interações do Usuário')
# dot.node('E', 'Usuário com Histórico?')
# dot.node('F', 'Exploitação (Recomendação Personalizada)')
# dot.node('G', 'Recomendar com SVD (Fatoração de Matrizes)')
# dot.node('H', 'Manter Exploração\n(10% Popularidade / Novidade)')
# dot.node('I', 'Ajustar para Recência (Decay Temporal)')
# 
# # Conexões
# dot.edge('A', 'B', label='Sim')
# dot.edge('A', 'E', label='Não')
# dot.edge('B', 'C')
# dot.edge('C', 'D')
# dot.edge('D', 'E')
# dot.edge('E', 'F', label='Sim')
# dot.edge('F', 'G')
# dot.edge('G', 'H')
# dot.edge('H', 'I', label='Equilíbrio Exploração/Exploitação')
# dot.edge('E', 'B', label='Não')
# 
# display(Image(dot.pipe(format='png')))

A recência é um componente do cálculo do RFV (Recência, Frequência e Valor), uma métrica de vendas que ajuda empresas a identificar os seus clientes mais importantes.
Como calcular a recência?
Calcula-se quantos dias se passaram desde a última compra do cliente

In [3]:
import glob
import pandas as pd

In [4]:
user_infos = pd.concat([pd.read_csv(fpath) for fpath in glob.glob('../data/raw/files/treino/*.csv')])
user_infos.head()

,userId,userType,historySize,history,timestampHistory,numberOfClicksHistory,timeOnPageHistory,scrollPercentageHistory,pageVisitsCountHistory,timestampHistory_new
0,f98d1132f60d46883ce49583257104d15ce723b3bbda21...,Non-Logged,3,"c8aab885-433d-4e46-8066-479f40ba7fb2, 68d2039c...","1657146417045, 1657146605778, 1657146698738","76, 38, 41","20380, 21184, 35438","50.3, 18.18, 16.46","2, 1, 1","1657146417045, 1657146605778, 1657146698738"
1,2c1080975e257ed630e26679edbe4d5c850c65f3e09f65...,Non-Logged,60,"3325b5a1-979a-4cb3-82b6-63905c9edbe8, fe856057...","1656684240278, 1656761266729, 1656761528085, 1...","7, 80, 2, 1, 7, 62, 26, 44, 4, 4, 14, 45, 13, ...","6049, 210489, 8672, 10000, 30000, 123007, 9965...","25.35, 45.66, 35.3, 28.05, 36.53, 47.57, 55.33...","1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1...","1656684240278, 1656761266729, 1656761528085, 1..."
2,0adffd7450d3b9840d8c6215f0569ad942e782fb19b805...,Logged,107,"04756569-593e-4133-a95a-83d35d43dbbd, 29b6b142...","1656678946256, 1656701076495, 1656701882565, 1...","0, 0, 0, 0, 0, 44, 0, 0, 2, 1, 0, 0, 0, 44, 0,...","311274, 140000, 32515, 157018, 118689, 159243,...","67.58, 47.22, 41.52, 63.09, 51.38, 65.11, 71.9...","1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1...","1656678946256, 1656701076495, 1656701882565, 1..."
3,c1e8d644329a78ea1f994292db624c57980b2886cfbc2d...,Non-Logged,56,"1f2b9c2f-a2d2-4192-b009-09065da8ec23, 04756569...","1658333312180, 1658404553818, 1658408449062, 1...","8, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 2, 0, 1, 1...","182696, 91925, 30000, 273655, 126409, 42980, 1...","58.26, 72.66, 22.57, 59.89, 40.36, 36.35, 14.7...","1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1...","1658333312180, 1658404553818, 1658408449062, 1..."
4,e777d1f31d4d955b63d60acc13df336d3903f52ab8f8f4...,Non-Logged,4,"bebdeb3e-1699-43e0-a1b8-989f5a6ab679, f4b484a7...","1658766608801, 1658766608801, 1660084035094, 1...","579, 579, 7, 2","801396, 801396, 10000, 10000","78.74, 78.74, 16.71, 9.34","7, 7, 1, 1","1658766608801, 1658766608801, 1660084035094, 1..."


In [5]:
user_historys = user_infos[['userId', 'history','timestampHistory','numberOfClicksHistory','timeOnPageHistory','scrollPercentageHistory','pageVisitsCountHistory','timestampHistory_new']]

user_historys = user_historys.set_index('userId').apply(lambda row: row.str.split(','), axis=1)
user_historys = user_historys.apply(pd.Series.explode).reset_index()
user_historys['numberOfClicksHistory'] = user_historys['numberOfClicksHistory'].astype(int)
user_historys['timeOnPageHistory'] = user_historys['timeOnPageHistory'].astype(int)
user_historys['scrollPercentageHistory'] = user_historys['scrollPercentageHistory'].astype(float)
user_historys['timestampHistory'] = user_historys['timestampHistory'].astype(int)
user_historys['pageVisitsCountHistory'] = user_historys['pageVisitsCountHistory'].astype(int)

user_historys.head()

,userId,history,timestampHistory,numberOfClicksHistory,timeOnPageHistory,scrollPercentageHistory,pageVisitsCountHistory,timestampHistory_new
0,f98d1132f60d46883ce49583257104d15ce723b3bbda21...,c8aab885-433d-4e46-8066-479f40ba7fb2,1657146417045,76,20380,50.30,2,1657146417045
1,f98d1132f60d46883ce49583257104d15ce723b3bbda21...,68d2039c-c9aa-456c-ac33-9b2e8677fba7,1657146605778,38,21184,18.18,1,1657146605778
2,f98d1132f60d46883ce49583257104d15ce723b3bbda21...,13e423ce-1d69-4c78-bc18-e8c8f7271964,1657146698738,41,35438,16.46,1,1657146698738
3,2c1080975e257ed630e26679edbe4d5c850c65f3e09f65...,3325b5a1-979a-4cb3-82b6-63905c9edbe8,1656684240278,7,6049,25.35,1,1656684240278
4,2c1080975e257ed630e26679edbe4d5c850c65f3e09f65...,fe856057-f97d-419f-ab1c-97c5c3e0719c,1656761266729,80,210489,45.66,1,1656761266729


In [6]:
news_item = pd.concat([pd.read_csv(fpath) for fpath in glob.glob('../data/raw/itens/itens/*.csv')])
news_item.head()

,page,url,issued,modified,title,body,caption
0,13db0ab1-eea2-4603-84c4-f40a876c7400,http://g1.globo.com/am/amazonas/noticia/2022/0...,2022-06-18 20:37:45+00:00,2023-04-15 00:02:08+00:00,Caso Bruno e Dom: 3º suspeito tem prisão tempo...,"Após audiência de custódia, a Justiça do Amazo...",Jeferson da Silva Lima foi escoltado por agent...
1,92907b73-5cd3-4184-8d8c-e206aed2bf1c,http://g1.globo.com/pa/santarem-regiao/noticia...,2019-06-20 17:19:52+00:00,2023-06-16 20:19:15+00:00,Linguajar dos santarenos é diferenciado e chei...,Vista aérea de Santarém\nÁdrio Denner/ AD Prod...,As expressões santarenas não significam apenas...
2,61e07f64-cddf-46f2-b50c-ea0a39c22050,http://g1.globo.com/mundo/noticia/2022/07/08/e...,2022-07-08 08:55:52+00:00,2023-04-15 04:25:39+00:00,Ex-premiê Shinzo Abe morre após ser baleado no...,Novo vídeo mostra que assassino de Shinzo Abe ...,Ex-primeiro-ministro foi atingido por tiros de...
3,30e2e6c5-554a-48ed-a35f-6c6691c8ac9b,http://g1.globo.com/politica/noticia/2021/09/0...,2021-09-09 19:06:46+00:00,2023-06-07 17:44:54+00:00,"Relator no STF, Fachin vota contra marco tempo...","Relator no STF, Fachin vota contra marco tempo...",Ministro defendeu que posse indígena é diferen...
4,9dff71eb-b681-40c7-ac8d-68017ac36675,http://g1.globo.com/politica/noticia/2021/09/1...,2021-09-15 19:16:13+00:00,2023-06-07 17:43:39+00:00,"\nApós 2 votos, pedido de vista suspende julga...",Após um pedido de vista (mais tempo para análi...,"Pelo marco temporal, índios só podem reivindic..."


# Premeira tentativa de recomendação heurística

In [7]:
from datetime import datetime
import pandas as pd
from scipy.sparse import csr_matrix
import numpy as np
from sklearn.preprocessing import MinMaxScaler



def calc_recency_score(issued: pd.Series):
    """
    Calcula o score de recência para uma série de timestamps.
    
    O score é inversamente proporcional ao tempo decorrido, 
    dando mais prioridade a notícias mais recentes.
    
    :param issued: Série com os timestamps das notícias.
    :return: Série com os scores de recência.
    """
    # Converter os timestamps para milissegundos

    now = parse_date_timestamp(datetime.timestamp(datetime.now()))

    # Calcular o score de recência, evitando divisão por zero
    return 1 / np.maximum(now - parse_date_timestamp(issued), 1e-6)


def parse_date_timestamp(date):
    # Converte para datetime se ainda for string
    if isinstance(date, str):
        date = pd.to_datetime(date, errors='coerce')
    elif isinstance(date, float):
        return (date * 1000)

    return int(datetime.timestamp(date) * 1000)


def history_by(userId: str):
    return user_historys.query('userId == @userId').set_index('history')

def user_by(userId: str):
    return user_infos.query('userId == @userId')

def created_sparse_matriz(df):
    user_id_category = df['userId'].astype('category')
    history_id_category = df['history'].astype('category')

    row = user_id_category.cat.codes
    col = history_id_category.cat.codes

    data = df['interaction_score']

    # Construindo a matriz esparsa com csr_matrix
    interaction_matrix = csr_matrix((data, (row, col)), shape=(len(df['userId'].unique()), len(df['history'].unique())))

    return user_id_category,history_id_category, interaction_matrix


# Criando score de recencia e score de populariadade por noticia

In [8]:
scaler = MinMaxScaler()


count_visits_by_news = (user_historys
                        .groupby('history')['pageVisitsCountHistory']
                        .count())
news_item['recency_score'] = scaler.fit_transform(
    news_item['issued'].apply(calc_recency_score).values.reshape(-1, 1)
)

news_item = (news_item.set_index('page').join(count_visits_by_news, how='inner').reset_index())
news_item.rename(columns={'pageVisitsCountHistory': 'popularity_score'}, inplace=True)
news_item.rename(columns={'index': 'history'},inplace=True)
news_item.head()

,history,url,issued,modified,title,body,caption,recency_score,popularity_score
0,92907b73-5cd3-4184-8d8c-e206aed2bf1c,http://g1.globo.com/pa/santarem-regiao/noticia...,2019-06-20 17:19:52+00:00,2023-06-16 20:19:15+00:00,Linguajar dos santarenos é diferenciado e chei...,Vista aérea de Santarém\nÁdrio Denner/ AD Prod...,As expressões santarenas não significam apenas...,0.271320,1
1,61e07f64-cddf-46f2-b50c-ea0a39c22050,http://g1.globo.com/mundo/noticia/2022/07/08/e...,2022-07-08 08:55:52+00:00,2023-04-15 04:25:39+00:00,Ex-premiê Shinzo Abe morre após ser baleado no...,Novo vídeo mostra que assassino de Shinzo Abe ...,Ex-primeiro-ministro foi atingido por tiros de...,0.948561,62
2,30e2e6c5-554a-48ed-a35f-6c6691c8ac9b,http://g1.globo.com/politica/noticia/2021/09/0...,2021-09-09 19:06:46+00:00,2023-06-07 17:44:54+00:00,"Relator no STF, Fachin vota contra marco tempo...","Relator no STF, Fachin vota contra marco tempo...",Ministro defendeu que posse indígena é diferen...,0.647653,1
3,9dff71eb-b681-40c7-ac8d-68017ac36675,http://g1.globo.com/politica/noticia/2021/09/1...,2021-09-15 19:16:13+00:00,2023-06-07 17:43:39+00:00,"\nApós 2 votos, pedido de vista suspende julga...",Após um pedido de vista (mais tempo para análi...,"Pelo marco temporal, índios só podem reivindic...",0.652242,1
4,a9fd6d34-6f40-4c90-849b-2ad36f04fd6f,http://g1.globo.com/politica/noticia/2021/08/2...,2021-08-27 18:49:03+00:00,2023-06-07 17:46:42+00:00,O que é o marco temporal sobre terras indígena...,Marco temporal sobre terras indígenas: entenda...,Tese analisada pelo Supremo após recurso da Fu...,0.637859,7


# Preparando dados para treinamento do modelo

In [9]:
import pandas as pd

# Obtém apenas os usuários logados para recomendação
users_info_logged = user_infos.query("userType == 'Logged'")[['userId']].set_index('userId')

# 🔗 Juntar os dados do histórico com as características dos itens
merged = pd.merge(user_historys, news_item, on='history').join(users_info_logged, on='userId', how='inner')
print(len(merged))
scaler = MinMaxScaler()

merged[['numberOfClicksHistory', 'pageVisitsCountHistory',
        'timeOnPageHistory', 'scrollPercentageHistory']] = scaler.fit_transform(
    merged[['numberOfClicksHistory', 'pageVisitsCountHistory',
            'timeOnPageHistory', 'scrollPercentageHistory']]
)

# 🎯 Cálculo do interaction_score utilizando apenas as colunas disponíveis
merged['interaction_score'] = (
        merged['numberOfClicksHistory'] * 0.4 +
        merged['pageVisitsCountHistory'] * 0.3 +
        merged['timeOnPageHistory'] * 0.2 +
        merged['scrollPercentageHistory'] * 0.1
)



119086


# Treinando o modelo

In [10]:
from sklearn.decomposition import TruncatedSVD

#em vez de usar pivot_table foi notado um alto custo de desempenho por isso ele foi substituido pelo csr_matrix
# # 🧱 Construção da matriz de interação
# interaction_matrix = merged.pivot_table(
#     index='userId',
#     columns='history',
#     values='interaction_score',
#     fill_value=0
# )

user_id_category, history_id_category,interaction_matrix =  created_sparse_matriz(merged)

# 📉 Aplicação do SVD
svd = TruncatedSVD(n_components=2)
user_factors = svd.fit_transform(interaction_matrix)
item_factors = svd.components_.T

# Criando a função de recomendação

In [11]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def recomender(user_id,historySize,history_size_min_limit=50, top_k=5, top_p=None):
    
    if historySize > history_size_min_limit:
        
        # Se top_p for None, usar todos os itens
        top_p = min(top_p or len(item_factors), len(item_factors))
    
        try:
            # 🆔 Obter o índice numérico do usuário
            user_index = user_id_category.cat.categories.get_loc(user_id)
    
            # 🧠 Obter o vetor latente do usuário
            user_vector = user_factors[user_index].reshape(1, -1)
    
            # 🧮 Calcular a similaridade com os top_p itens
            similarities = cosine_similarity(user_vector, item_factors).flatten()
    
            # 🔝 Selecionar os índices dos itens mais similares dentro do top_p
            recommended_indices = similarities.argsort()[::-1][:top_p]
    
            # 🎯 Ajustar os índices para o mapeamento correto nos itens originais
            recommended_items = history_id_category.cat.categories[recommended_indices]
    
        except KeyError:
            print(f"User ID {user_id} não encontrado! O usuário pode estar offline")
            return []
    
        # 📊 Retornar os itens recomendados, agora incluindo a similaridade
        return  (
            news_item.set_index('history')
            .loc[recommended_items]
            .assign(similarity=similarities[recommended_indices])
            .sort_values(['similarity', 'recency_score', 'popularity_score'], ascending=[False, False, False])
            .head(top_k)
        )   
    
    else:
        return (
            news_item.set_index('history')
            .sort_values(['popularity_score', 'recency_score'], ascending=[False, False])
            .head(top_k)
        )


In [12]:
recomender(
    user_id='e2ea40933f782f036277b6e250eb26a9d5e30ac17069806d206bcd48bcd7fa59',
    historySize=60,
    top_k=5)

# 749
# 1647
#628
#202
#1023

,url,issued,modified,title,body,caption,recency_score,popularity_score,similarity
d3b94bea-1139-4357-b5cc-879569870c94,http://g1.globo.com/rj/rio-de-janeiro/noticia/...,2020-05-14 15:35:13+00:00,2020-05-14 16:20:18+00:00,Mulher é atingida na cabeça por pneu em calçad...,Mulher é atingida na cabeça por pneu quando ca...,Vítima caminhava pela calçada quando pneu sai ...,0.381681,36,1.0
b74a9281-7372-47ad-93da-9d264d4c7afb,http://g1.globo.com/economia/noticia/2020/11/1...,2020-11-16 15:14:01+00:00,2020-11-17 10:13:12+00:00,PIS-Pasep 2020-2021: abono salarial começa a s...,O abono salarial PIS-Pasep 2020-2021 começa a ...,"Só no PIS, mais de 686 mil trabalhadores nasci...",0.464619,422,1.0
6e7f40f2-8c3a-499b-8e0b-63bd05a4359d,http://g1.globo.com/musica/noticia/viuva-de-ch...,2017-09-07 13:53:04+00:00,2017-09-07 13:53:07+00:00,Viúva de Chester Bennington publica foto do ca...,Viúva de Chester Bennington publica foto do ca...,Foto foi tirada dias antes do vocalista do Lin...,0.131265,52,1.0
8458c0c6-5e5d-4e50-a5f9-6aea23ac3ff8,http://g1.globo.com/fantastico/noticia/2022/07...,2022-07-17 23:41:46+00:00,2022-07-17 23:41:46+00:00,"'Já não sentia meus pés', diz caminhoneiro que...",Brasileiros ficam presos em nevasca\nTudo ia b...,O Fantástico traz o relato de brasileiros que ...,0.961276,14,1.0
7f631ad8-46af-432b-967b-de58a58d58ef,http://g1.globo.com/am/amazonas/noticia/motori...,2017-06-22 22:40:52+00:00,2017-06-22 22:40:52+00:00,Motorista tem pena reduzida por acidente que m...,Corpo de Bombeiros fez o resgate das 7 vítimas...,"Segundo o TJ, pena foi reduzida após apelação ...",0.119055,95,1.0


In [13]:
user_infos.query("historySize >50 and userType=='Logged'").head(2)

,userId,userType,historySize,history,timestampHistory,numberOfClicksHistory,timeOnPageHistory,scrollPercentageHistory,pageVisitsCountHistory,timestampHistory_new
2,0adffd7450d3b9840d8c6215f0569ad942e782fb19b805...,Logged,107,"04756569-593e-4133-a95a-83d35d43dbbd, 29b6b142...","1656678946256, 1656701076495, 1656701882565, 1...","0, 0, 0, 0, 0, 44, 0, 0, 2, 1, 0, 0, 0, 44, 0,...","311274, 140000, 32515, 157018, 118689, 159243,...","67.58, 47.22, 41.52, 63.09, 51.38, 65.11, 71.9...","1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1...","1656678946256, 1656701076495, 1656701882565, 1..."
12,1859975e492157269a52ca0981bebe6479613a5e378130...,Logged,90,"3c787cdc-99f7-4001-94ac-4edb2e971b94, aeab0e46...","1656688881587, 1656887307494, 1656952061733, 1...","10, 9, 0, 0, 0, 5, 0, 1, 1, 9, 0, 18, 5, 6, 18...","50000, 33323, 8033, 60611, 57376, 140000, 3228...","53.75, 50.79, 40.78, 60.74, 42.18, 64.21, 63.7...","1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1...","1656688881587, 1656887307494, 1656952061733, 1..."


In [14]:
user_by('e2ea40933f782f036277b6e250eb26a9d5e30ac17069806d206bcd48bcd7fa59')

,userId,userType,historySize,history,timestampHistory,numberOfClicksHistory,timeOnPageHistory,scrollPercentageHistory,pageVisitsCountHistory,timestampHistory_new
303,e2ea40933f782f036277b6e250eb26a9d5e30ac1706980...,Logged,1,d3b94bea-1139-4357-b5cc-879569870c94,1657308472385,0,11068,40.61,1,1657308472385


# Salvando dados no datawarehorse

In [15]:
from sqlalchemy import create_engine

# 💾 Salvar no SQLite
engine = create_engine('sqlite:///../data/refined/datawarehouse.db', echo=False)

# Salvar os DataFrames no banco de dados
merged.to_sql('merged_data', con=engine, if_exists='replace', index=False)
news_item.to_sql('news_item', con=engine, if_exists='replace', index=False)

78664